<a href="https://colab.research.google.com/github/StratosFair/Mean_Escape_Time/blob/main/OU_process_2D/Models/adapted_pinn_medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Imports & FP64
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.special as special

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_dtype(torch.float64)

# Solving for the MET of an Ornstein-Uhlenbeck process in a disk with "Adapted Architecture PINNs" : comparison with exact solution

## 1) Setup

For $\theta,\sigma,r >0$ some fixed parameters, we let $\Omega := B_r \equiv \{x\in\mathbb R^d : \|x\|< r \}$, and define the process ($d=2$ in our illustration) :

$$\begin{cases} dX_t &= -\theta X_t dt + \sigma dB_t \\
X_0 &= x \in \Omega \end{cases} $$

For all $x\in\Omega$, let

$$T(x) := \inf\{t\ge 0 : X_t \in\partial\Omega\} $$

and let its first moment be denoted

$$\tau(x) := \mathbb E[T(x)] $$

We can show under some regularity conditions on $\Omega$ that $\tau$ is the (unique) solution of the BVP :

$$\begin{cases} -\mathcal{L}u(x) &= 1 \text{ for all } x\in\Omega \\
u(x) &= 0 \text{ for all } x\in\partial\Omega \end{cases} $$

where $\mathcal L$ is the infinitesimal generator of the Ornstein-Uhlenbeck process, given by
$$\mathcal Lu : x \mapsto -\theta x \cdot \nabla u(x) + \frac{\sigma^2}{2}\Delta u(x) $$

Nicely enough, for this problem, we can compare our solution with the known closed-form solution (see https://arxiv.org/abs/2208.04029) :

$$ \tau(x) := \frac{1}{\lambda^{d/2}\sigma^2}\int_\rho^r z^{1-d} e^{\lambda z^2} \gamma(d/2, \lambda z^2)\ dz $$

where $\lambda := \theta/\sigma^2 $, $\rho := \|x\| $ and $\gamma$ is the upper incomplete gamma function :
$$\gamma(n,y) := \int_0^y t^{n-1} e^{-t}\ dt.  $$

In [15]:
# Problem Parameters & True Solution
R = 2.0
theta = 1.0
sigma = 1.0

def true_tau(x, y, theta=theta, sigma=sigma, R=R):
    lam = theta / sigma**2
    rho = np.sqrt(x**2 + y**2)
    integrand = lambda t: np.exp(lam * t**2) * special.gammainc(1, lam * t**2) / t
    I, _ = integrate.quad(integrand, rho, R)
    return I / theta

true_tau_vec = np.vectorize(true_tau)

## 2) Neural Network architecture and loss function

Unlike the standard PINN, we will take an hypothesis space of Neural Networks which satisfy the boundary conditions explicitly. For this problem, we have homogenous zero Dirichlet boundary conditions, which we can enforce by multiplying our Neural Networks with an appropriate "smooth distance approximation" (see https://arxiv.org/abs/2104.08426). In accordance with the mentioned paper, we will take
$$\varphi : x\mapsto \frac{r^2 - \|x\|^2}{2r}$$
as our smooth distance approximation.  


With this modification, our objective to minimize becomes
$$\hat u := \arg\min_{u\in\mathcal{NN}_\varphi}\ \frac1n \sum_{i=1}^n (\mathcal L u(x_i^c) + 1)^2 $$
where $x_i^c$ are sampled i.i.d. with uniform distribution on $\Omega$,
$$\mathcal{NN}_\varphi:=\left\{x\mapsto \varphi(x) \cdot T_L\circ \sigma \circ T_{L-1}\circ\cdots\circ \sigma\circ T_1 (x)\right\}, $$
for
$$T_\ell : \mathbb{R}^{\ell-1}\to\mathbb{R}^\ell $$
affine-linear maps of appropriate input-output dimensions, where
$$\sigma :x \mapsto \begin{cases}x^2 &\text{ if } x\ge 0\\ 0 &\text{ if } x\le 0\end{cases} \quad \text{OR }\ \ \sigma : x\mapsto \operatorname{tanh}(x),$$
is either the ReQU or hyperbolic tangent activation function, which is understood element-wise when applied to vectors.

In [16]:
# defining PINN w/ matching boundary condition
power = 2 #exponent k for relu^k, none for tanh
width = 50
depth = 3
gain = 1.0 #magnitude of weights at initialization

#define ReLU^k activation

class RePU(nn.Module):
    def __init__(self, power = power):
        super(RePU, self).__init__()
        self.power = power

    def forward(self, x):
        return torch.pow(torch.relu(x), self.power)

#define the smooth distance approximation
def smooth_distance(x):
    norm_x = torch.linalg.norm(x, dim=-1)
    return (R**2 - norm_x**2)/(2*R)

#define hypothesis space
class BoundaryPINN(nn.Module):
    def __init__(self, power = power, width = width, depth = depth):
        super(BoundaryPINN,self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(2, width))
        if isinstance(power, int):
            self.layers.append(RePU(power))
            for _ in range(depth-1) :
                self.layers.append(nn.Linear(width, width))
                self.layers.append(RePU(power))
        else :
            self.layers.append(nn.Tanh())
            for _ in range(depth-1) :
                self.layers.append(nn.Linear(width, width))
                self.layers.append(nn.Tanh())
        self.layers.append(nn.Linear(width, 1))

        #for dynamic weighting, first biased towards data fidelity
        self.log_sigma_pde = nn.Parameter(torch.tensor(0.0))
        self.log_sigma_data = nn.Parameter(torch.tensor(-np.log(2),\
                                                        dtype=torch.float64))

    def forward(self, x):
        output = x
        for layer in self.layers:
            output = layer(output)
        distance =  smooth_distance(x)
        return output * distance.unsqueeze(-1)

#weight initialization
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_normal_(m.weight, gain=gain)
        m.bias.data.fill_(gain)

In [17]:
# defining PDE Residual & Loss, enabling grad on x
def pde_residual(model, x, theta, sigma, forcing = None):
    # ensure x is on the right device, float64, and requires grad
    x = x.clone().detach().to(device).requires_grad_(True)

    # forward through PINN
    tau = model(x)                                # [N]
    # first derivatives
    grads = torch.autograd.grad(
        tau, x,
        grad_outputs=torch.ones_like(tau),
        create_graph=True,
    )[0]                                        # [N,2]
    tau_x, tau_y = grads[:,0], grads[:,1]

    # second derivatives
    tau_xx = torch.autograd.grad(
        tau_x, x,
        grad_outputs=torch.ones_like(tau_x),
        create_graph=True
    )[0][:,0]
    tau_yy = torch.autograd.grad(
        tau_y, x,
        grad_outputs=torch.ones_like(tau_y),
        create_graph=True
    )[0][:,1]

    # OU‐generator L[tau]
    phys = -theta*(x[:,0]*tau_x + x[:,1]*tau_y) \
           + 0.5*sigma**2*(tau_xx + tau_yy)

    if forcing is None:
        # default constant −1
        f_vals = -1.0
    elif isinstance(forcing, (int, float)):
        # constant forcing = that number
        f_vals = float(forcing)
    elif callable(forcing):
        # call it on x
        f_vals = forcing(x)
        # flatten to [N]
        if f_vals.dim() > 1:
            f_vals = f_vals.view(-1)
    else:
        raise ValueError(f"forcing must be None, float, or callable, got {type(forcing)}")

    # turn any scalar into a tensor of shape [N]
    if isinstance(f_vals, float) or isinstance(f_vals, int):
        f_vals = x.new_full((x.shape[0],), float(f_vals))

    # 7) residual = L[tau] - f(x)
    r = phys - f_vals              # shape [N]
    return r

def loss_fn(res):
    return torch.mean(res.pow(2))

# 3) Computing an approximate data penalty regularization

We sample a small subset $x_1, \ldots, x_{n_{data}} \in \Omega$ of points for which we compute approximate values $\tau^{MC}(x_1), \ldots, \tau^{MC}(x_{n_{data}}) $ by Monte Carlo. we will then use them to define a "data-fidelity" penalty term in the loss function:
$$\text{Penalty}(\hat\tau_{NN}) = \frac{\lambda_{data}}{n_{data}} \sum_{i=1}^n \left(\tau^{MC}(x_i) - \hat\tau_{NN}(x_i)\right)^2, $$
where $\lambda_{data}$ is a positive constant.

In [18]:
# generating some approximate true solutions in the domain

#sampling anchor points, biased towards center (failure zone)
def sample_disk(batch_size, R, radius_frac=0.6):
    """
    Sample uniformly in the DISK of radius (radius_frac * R).
    This is the simplest way to concentrate points near the centre.
    """
    r = torch.sqrt(torch.rand(batch_size, device=device))  # uniform‐disk radius ∝ sqrt(U)
    theta = 2 * torch.pi * torch.rand(batch_size, device=device)
    x = torch.stack([r * torch.cos(theta), r * torch.sin(theta)], dim=1)
    return radius_frac * R * x  # now uniform in disk of radius radius_frac*R


# 2) monte-carlo (euler-maruyama) approximation of tau at these points
@torch.no_grad()
def mc_exit_time(x0, theta, sigma, R,
                 n_paths=128, dt=1e-3, max_steps=20000):
    """
    x0: [B,2] starting points
    Returns tau_hat: [B] = mean exit time from each x0.
    Any path still 'alive' after max_steps is counted as having exit time = max_steps*dt.
    """
    B = x0.shape[0]
    P = n_paths

    # Expand to per‐path positions
    X = x0.unsqueeze(1).expand(B, P, 2).clone()  # [B,P,2]
    t = torch.zeros(B, P, device=device)         # running clock
    alive = torch.ones(B, P, dtype=torch.bool, device=device)
    sqrt_dt = sigma * (dt ** 0.5)

    for step in range(max_steps):
        if not alive.any():
            break

        # Drift + noise, but only update those still alive
        X_alive = X[alive]
        drift    = -theta * X_alive * dt
        noise    = sqrt_dt * torch.randn_like(X_alive)
        X[alive] += drift + noise
        t[alive] += dt

        # Mark newly‐exited paths
        just_exited = (X[alive].pow(2).sum(dim=1) >= R*R)
        idx_alive   = alive.nonzero(as_tuple=False)  # [[i1,j1], [i2,j2], ...]
        exited_idx  = idx_alive[just_exited]
        alive[exited_idx[:,0], exited_idx[:,1]] = False

    # Paths still alive get t = max_steps*dt automatically from the loop
    tau_hat = t.mean(dim=1)  # [B]
    return tau_hat  # no gradient flows back

# 4) Homotopy Regularization

We define a smooth function $g(x)$ for regularizing the constant forcing term as

$$
g(x) = \mathrm{magnitude} \cdot \sigma\!\left(\frac{1}{\sigma_{\mathrm{raw}}}\left[\sum_{j=1}^{M} a_j\cos\Big(\langle \omega_j, x \rangle + b_j\Big) - \mu\right]\right)
\cdot \frac{\max\{R^2 - \|x\|^2,\,0\}}{2R},
$$

where

- $a_j$, $\omega_j$, and $b_j$ are random coefficients (with $\omega_j$ scaled by a base frequency, and $b_j$ uniformly drawn from $[0,2\pi]$),
- The **raw Fourier sum** is
  $$
  G_{\mathrm{raw}}(x) = \sum_{j=1}^{M} a_j\cos\!\Big(\langle \omega_j, x \rangle + b_j\Big),
  $$
  which is then normalized by subtracting its mean $\mu$ and dividing by its standard deviation $\sigma_{\mathrm{raw}}$;
- $\sigma(\cdot)$ is the sigmoid function that bounds the output to $(0,1)$;
- The radial mask
  $$
  \frac{\max\{R^2 - \|x\|^2,\,0\}}{2R}
  $$
  ensures that $g(x)$ smoothly decays to zero as $\|x\|$ approaches $R$.

**Quick Intuition:** The function combines a randomized Fourier series with normalization, a sigmoid activation, and a radial decay. This results in a smooth, bounded perturbation to the forcing term, enhancing the regularity needed for improved convergence of the PINN solver.

In [19]:
# Random Fourier Homotopy g(x) to regularize the forcing term
class RandomFourierHomotopy:
    def __init__(self, M=200, freq_scale=None, R=R, magnitude=8.0):
        """
        M           : number of random Fourier modes
        freq_scale  : base frequency scale (if None, defaults to π/R)
        R           : radius of the disk
        magnitude   : overall output scale
        """
        self.M         = M
        self.R         = R
        self.magnitude = magnitude

        if freq_scale is None:
            # pick a lower base frequency so the field is smooth
            freq_scale = math.pi / R

        # sample random directions & phases
        # omega: [M,2], b: [M], a: [M]
        self.omega = torch.randn(M, 2) * freq_scale
        self.b     = 2 * math.pi * torch.rand(M)
        # normalize by sqrt(M) so that raw variance ~1
        self.a     = torch.randn(M) / math.sqrt(M)

    def __call__(self, x):
        """
        x: [N,2] tensor of (x,y) points
        returns: [N] tensor of forcing values
        """
        # 1) compute raw random Fourier sum
        #    proj: [N,M] = x·omega^T + b
        proj  = x @ self.omega.t() + self.b
        g_raw = (self.a * torch.cos(proj)).sum(dim=1)    # [N]

        # 2) normalize to zero mean, unit std dev (over this batch)
        mean   = g_raw.mean()
        stddev = g_raw.std(unbiased=False).clamp(min=1e-6)
        g_norm = (g_raw - mean) / stddev                # [N]

        # 3) nonlinearity: tanh keeps it in (–1,1) but nonzero everywhere
        g_nl = torch.sigmoid(g_norm)                       # [N]

        # 4) radial mask to fade to zero at |x|=R
        r2   = (x**2).sum(dim=1)                        # [N]
        mask = ((self.R**2 - r2).clamp(min=0)
                / (2 * self.R))                       # [N]

        # 5) scale to the desired magnitude
        return self.magnitude * g_nl * mask             # [N]

# 5) Adaptive sampling [TODO: add citation]
The function `make_adaptive_batch` creates a mini-batch that focuses training on areas with high PDE residuals:

1. **Residual Computation:**  
   A candidate pool `Xp` is sampled uniformly over the disk. For each point $x$, the PDE residual is computed via  
   $$
   r(x) = \texttt{pde_residual(model, x, theta, sigma, forcing)}
   $$  
   and the absolute residual is defined as  
   $$
   R(x) = |r(x)|.
   $$

2. **Adaptive Selection:**  
   A fraction of the batch (defined by `hard_frac`) is formed using high-error points. There are two modes:
   - **Hard Sampling:** Directly select the top $K = \texttt{hard_frac} \times \texttt{batch_size}$ points with the highest $R(x)$.
   - **Soft Sampling:** Sample points with probability proportional to  
     $$
     p(x) \propto R(x)^\beta.
     $$
     The parameter $\beta$ controls the selection sharpness:
     - A small $\beta$ makes the sampling nearly uniform.
     - A large $\beta$ heavily favors points with high residuals.

3. **Batch Formation:**  
   The final batch is a combination of these adaptively chosen points and randomly selected points from the remaining pool.

This method helps concentrate training on regions where the model underperforms while still maintaining overall coverage.

In [20]:
# defining Uniform + Adaptive Sampling functions in the disk
def sample_uniform_disk(N, R=R):
    theta = 2*np.pi * torch.rand(N, dtype=torch.float64, device=device)
    rho = R    * torch.sqrt(torch.rand(N, dtype=torch.float64, device=device))
    return torch.stack([rho*torch.cos(theta), rho*torch.sin(theta)], dim=1)  # [N,2]

def make_adaptive_batch(
    model, pool_size, batch_size,
    forcing, theta, sigma,
    hard_frac=0.1,    # fraction of batch from highest residuals
    use_soft=False,   # if True, sample soft rather than top‐K
    beta=1.0          # exponent for soft sampling
):
    # candidate pool
    Xp = sample_uniform_disk(pool_size)

    # 2) Compute residuals _with_ gradient‐tracking so second‐derivatives work
    r_all = pde_residual(model, Xp, theta, sigma, forcing)  # [pool_size]
    # 3) Detach and take absolute
    Rvals = r_all.detach().abs()                             # [pool_size]

    # number of hard points
    K     = int(batch_size * hard_frac)
    if use_soft or beta < 10.0 :
        # 2a) soft sampling: probabilities ∝ (residual^beta)
        weights = Rvals.pow(beta)
        weights = weights / (weights.sum() + 1e-12)
        hard_idx = torch.multinomial(weights,
                                     num_samples=K,
                                     replacement=False)
    else:
        # 2b) hard sampling: pure top‐K
        hard_idx = torch.topk(Rvals, K).indices # [K]

    all_idx  = torch.arange(pool_size, device=device)
    mask     = torch.ones(pool_size, dtype=torch.bool, device=device)
    mask[hard_idx] = False
    rest_idx = all_idx[mask]
    n_rest   = batch_size - K
    rnd      = rest_idx[torch.randperm(rest_idx.numel(), device=device)[:n_rest]]

    batch_idx = torch.cat([hard_idx, rnd], dim=0)                # [batch_size]
    return Xp[batch_idx]                                      # [batch_size,2]

# 6) Training loop

 ## 6.1) Training Parameters

In [21]:
# Training parameters
pool_size        = 4096
batch_size       = 512
n_epochs         = 20_000
learning_rate    = 5e-2
beta = 1 # 0 -> no adaptive sampling, infty -> top K sampling


# model instantiation
model = BoundaryPINN().to(device)
model.apply(init_weights)

opt   = torch.optim.Adam(list(model.parameters()), lr=learning_rate)

# scheduler: every `step_size` epochs multiply lr by gamma
step_size = 500        # decay lr every step_size epochs
n_steps   = n_epochs // step_size
damping   = 1e-2
gamma     = math.exp(math.log(damping) / n_steps)        # mutiply lr by gamma at each step
scheduler = torch.optim.lr_scheduler.StepLR(opt,
                                            step_size=step_size,
                                            gamma=gamma)

## 6.2) Generating the "data-fidelity" samples

In [ ]:
# sample x_data
x_data = sample_disk(batch_size = batch_size//2,
                     R = R,
                     radius_frac = 0.8)

# precompute tau_data once (theta, sigma are fixed PDE params)
tau_data = mc_exit_time(
    x0       = x_data,
    theta    = theta,
    sigma    = sigma,
    R        = R,
    n_paths  = 256,
    dt       = 1e-3,
    max_steps= 10_000
)

## 6.3) Parametrizing the homotopy function and schedule

In [ ]:
# parametrizing the homotopy
sigma_true = sigma       # true diffusion coefficient
theta_true = theta       # true drift coefficient
r_true     = theta_true / sigma_true # true ratio

# "easy" and true coefficient ratio
r0 = 0.25 if r_true > 0.25 else r_true
r1 = r_true

# "easy" and true forcing term
g_hom = RandomFourierHomotopy(M=50)
f_0 = lambda x : -1.0 - g_hom(x)
f_1 = lambda x : x.new_full((x.shape[0],), -1.0)

#helper functions
nu_step = 0.1
def make_nu(epoch, n_hom, step = 0.1):
    """Returns nu in steps of `step` from 0 up to 1 after n_hom epochs."""
    if epoch >= n_hom:
        return 1.0
    continuous_nu = float(epoch) / n_hom
    discrete_nu = math.floor(continuous_nu / step) * step
    return discrete_nu

def interp(a0, a1, nu):
    return a0 * (1.0 - nu) + a1 * nu

In [ ]:
# plot homotopy forcing g_hom(x)
n_pts = 200
xx, yy = np.meshgrid(
    np.linspace(-R,R,n_pts),
    np.linspace(-R,R,n_pts)
)
mask = xx**2 + yy**2 <= R**2
grid_pts = np.vstack([xx[mask], yy[mask]]).T

with torch.no_grad():
    g_vals = g_hom(torch.from_numpy(grid_pts).to(device)).cpu().numpy()

Zg = np.zeros_like(xx)
Zg[mask] = g_vals

plt.figure(figsize=(5,5))
cg = plt.contourf(xx, yy, Zg, levels=50, cmap='coolwarm')
plt.colorbar(cg, shrink=0.8)
plt.title('Homotopy forcing $g_{\\rm hom}(x)$')
plt.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
def pde_residual_homotopy(model, X, nu):
    """
    Single‐nu homotopy:
      - theta_nu = (interp(r0,r1,nu)) * sigma_true
      - f_nu(x)  = interp( f_0(x), f_1(x), nu )
    If nu=None, uses theta_true and f_1 exactly.
    """
    # 1) drift/diff ratio
    if nu is None:
        theta_nu = theta_true
    else:
        r_nu     = interp(r0, r1, nu)
        theta_nu = r_nu * sigma_true

    # 2) forcing
    if nu is None:
        forcing_nu = f_1
    else:
        forcing_nu = lambda x : interp(f_0(x), f_1(x), nu)

    return pde_residual(model, X, theta_nu, sigma_true, forcing=forcing_nu)

## 6.4) Start the Training with Adam

In [ ]:
# Training Loop w/ Homotopy & Loss Logging

# prep for best‐model tracking
best_loss       = float("inf")
best_model_path = "best_adam.pth"
best_epoch = 0
eval_Xb = sample_uniform_disk(pool_size, R=R) #to evaluate the model

total_history = []
pde_history   = []
data_history  = []

n_hom =  int(0.6 * n_epochs)   # epochs to ramp nu from 0 to 1

print("\n=== Training with single‐nu homotopy ramp ===")
model.train()
for ep in range(1, n_epochs+1):
    nu = make_nu(ep, n_hom, nu_step)   # in [0,1], then stays at 1

    opt.zero_grad()

    # 2) build one adaptive interior batch using current nu
    if nu is None:
        forcing_nu = f_1
        theta_nu = theta_true
    else:
        forcing_nu = lambda x : interp(f_0(x), f_1(x), nu)
        r_nu     = interp(r0, r1, nu)
        theta_nu = r_nu * sigma_true

    Xb = make_adaptive_batch(
        model, pool_size, batch_size,
        forcing_nu, theta_nu, sigma_true, beta=beta
    )  # → [batch_size,2]

    # 3) PDE residual + loss
    r        = pde_residual_homotopy(model, Xb, nu)  # uses theta_nu, f_nu under the hood
    loss_pde = loss_fn(r)

    # 4) data‐loss (fixed x_data, tau_data)
    tau_pred  = model(x_data).squeeze(-1)
    loss_data = (tau_pred - tau_data).pow(2).mean()

    # 5) total + backward + step
    loss_total = torch.exp(-model.log_sigma_pde) * loss_pde \
               + torch.exp(-model.log_sigma_data) * loss_data \
               + model.log_sigma_pde + model.log_sigma_data

    # 6) backward + step
    loss_total.backward()
    opt.step()
    scheduler.step()

    #eval
    r_eval        = pde_residual_homotopy(model, eval_Xb , None)  #evaluate the model on the true PDE
    loss_eval     = torch.exp(-model.log_sigma_pde) * loss_fn(r_eval).item() \
                  + torch.exp(-model.log_sigma_data) * loss_data\
                  + model.log_sigma_pde + model.log_sigma_data

    if loss_eval < best_loss:
        best_loss = loss_eval
        torch.save(model.state_dict(), best_model_path)
        best_epoch = ep


    # 7) logging & histories
    total_history.append(loss_total.item())
    pde_history.append(loss_pde.item())
    data_history.append(loss_data.item())

    if ep % 100 == 0:
        print(
            f"[ep {ep:4d}/{n_epochs}] "
            f"nu={nu:.3f}  "
            f"loss_pde={loss_pde:.2e}  "
            f"loss_data={loss_data:.2e}"
        )
    if ep % 500 == 0:
        lr = scheduler.get_last_lr()[0]
        print(f"  → epoch {ep:4d},  lr={lr:.2e}")

## 6.5) L-BFGS Fine-Tuning

In [ ]:
#LBFGS fine-tuning

# Reload best Adam snapshot
print(f"\nLoading best Adam model (loss={best_loss:.2e}, epoch={best_epoch}) …")
model.load_state_dict(torch.load(best_model_path))
model.to(device)


# define L-BFGS parameters
lbfgs = torch.optim.LBFGS(
    model.parameters(),
    lr=1.0,
    max_iter=500,
    history_size=500,
    tolerance_grad=0,
    tolerance_change=0,
    line_search_fn="strong_wolfe"
)

# freeze one big PDE batch for the closure
Xb_ft = make_adaptive_batch(
    model, pool_size, pool_size,
    f_1, theta, sigma, beta=beta
).detach()

lbfgs_total = []
lbfgs_pde   = []
lbfgs_data  = []



#closure function
def closure():
    closure.calls += 1
    lbfgs.zero_grad()
    # PDE term
    r_ft    = pde_residual_homotopy(model, Xb_ft, nu=None)
    pde_l   = loss_fn(r_ft)
    # data term
    tau_p   = model(x_data).squeeze(-1)
    data_l  = (tau_p - tau_data).pow(2).mean()
    total_l = torch.exp(-model.log_sigma_pde) * pde_l \
            + torch.exp(-model.log_sigma_data) * data_l \
            + model.log_sigma_pde + model.log_sigma_data

    # record
    lbfgs_total.append(total_l.item())
    lbfgs_pde.append(  pde_l.item()   )
    lbfgs_data.append( data_l.item()   )

    total_l.backward()
    return total_l

closure.calls = 0

In [ ]:
# report losses before/after lbfgs
# make sure model is in eval mode so e.g. Dropout/BatchNorm won’t move
model.eval()

# --- PDE loss (needs grad to compute ∇u inside pde_residual) ---
r_before = pde_residual_homotopy(model, Xb_ft, nu=None)
before_pde = loss_fn(r_before).item()

# --- data loss (pure forward, safe under no_grad) ---
with torch.no_grad():
    pred = model(x_data).squeeze(-1)
    before_data = (pred - tau_data).pow(2).mean().item()

print(f"Before L-BFGS → PDE {before_pde:.2e}, DATA {before_data:.2e}")

# … run your L-BFGS …
print("→ Running L-BFGS …")
loss_after = lbfgs.step(closure)
print(f"L-BFGS did {closure.calls} closure calls, final total loss = {loss_after:.2e}")

# after L-BFGS, same pattern:
model.eval()
r_after = pde_residual_homotopy(model, Xb_ft, nu=None)
after_pde = loss_fn(r_after).item()
with torch.no_grad():
    pred = model(x_data).squeeze(-1)
    after_data = (pred - tau_data).pow(2).mean().item()
print(f" After L-BFGS → PDE {after_pde:.2e}, DATA {after_data:.2e}")

# 7) Plotting the results

## 7.1) Training curves

In [ ]:
# Plotting three separate loss evolutions
total_history = total_history + lbfgs_total
pde_history   = pde_history + lbfgs_pde
data_history  = data_history + lbfgs_data

# a) Total Loss
plt.figure(figsize=(6,4))
plt.semilogy(np.arange(n_hom), total_history[:n_hom], \
             color = 'blue', label = "ADAM with homotopy")
plt.semilogy(np.arange(n_hom, n_epochs), total_history[n_hom: n_epochs], \
             color = 'orange', label = "Adam on true PDE")
plt.semilogy(np.arange(n_epochs, len(total_history)), total_history[n_epochs:], \
             color = 'green', label = "L-BFGS on true PDE")
plt.title("Total Loss vs Epoch / L-BFGS step")
plt.xlabel("step (Adam epochs or L-BFGS iter)")
plt.ylabel("loss_total")
plt.legend()
plt.grid(True, which="both", ls="--", alpha=0.5)

# b) PDE Loss
plt.figure(figsize=(6,4))
plt.semilogy(np.arange(n_hom), pde_history[:n_hom], \
             color = 'blue', label = "ADAM with homotopy")
plt.semilogy(np.arange(n_hom, n_epochs), pde_history[n_hom: n_epochs], \
             color = 'orange', label = "Adam on true PDE")
plt.semilogy(np.arange(n_epochs, len(pde_history)), pde_history[n_epochs:], \
             color = 'green', label = "L-BFGS on true PDE")
plt.title("PDE Loss vs Epoch / L-BFGS step")
plt.xlabel("step")
plt.ylabel("loss_pde")
plt.legend()
plt.grid(True, which="both", ls="--", alpha=0.5)

# c) Data Loss
plt.figure(figsize=(6,4))
plt.semilogy(np.arange(n_hom), data_history[:n_hom], \
             color = 'blue', label = "ADAM with homotopy")
plt.semilogy(np.arange(n_hom, n_epochs), data_history[n_hom: n_epochs], \
             color = 'orange', label = "Adam on true PDE")
plt.semilogy(np.arange(n_epochs, len(pde_history)), data_history[n_epochs:], \
             color = 'green', label = "L-BFGS on true PDE")
plt.title("Data Loss vs Epoch / L-BFGS step")
plt.xlabel("step")
plt.ylabel("loss_data")
plt.legend()
plt.grid(True, which="both", ls="--", alpha=0.5)

plt.show()

## 7.2) Learned Solution vs True Solution

In [ ]:
# 9) Visualize PINN vs True tau

n_pts = 200
xx, yy = np.meshgrid(
    np.linspace(-R,R,n_pts),
    np.linspace(-R,R,n_pts)
)
mask = xx**2 + yy**2 <= R**2
pts = np.vstack([xx[mask], yy[mask]]).T

with torch.no_grad():
    inp = torch.from_numpy(pts).to(device)
    pred = model(inp).cpu().numpy().squeeze(-1)

Zp = np.zeros_like(xx); Zt = np.zeros_like(xx)
Zp[mask], Zt[mask] = pred, true_tau_vec(xx[mask], yy[mask])

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,5))
cf1 = ax1.contourf(xx, yy, Zp, levels=50, cmap='viridis')
ax1.set_title('PINN tau')
fig.colorbar(cf1, ax=ax1, shrink=0.8)

cf2 = ax2.contourf(xx, yy, Zt, levels=50, cmap='viridis')
ax2.set_title('True tau')
fig.colorbar(cf2, ax=ax2, shrink=0.8)
plt.tight_layout()
plt.show()

## 7.3) Relative error

In [ ]:
# average relative L2 error
#    ‖tau_pinn − tau_true‖₂ / ‖tau_true‖₂
pred_vals = pred
true_vals = true_tau_vec(pts[:,0], pts[:,1])
tau_l2_norm =  np.linalg.norm(true_vals)
glob_rel_L2 = np.linalg.norm(pred_vals - true_vals) / tau_l2_norm
print(f"Global relative L2 error: {glob_rel_L2:.3e}")

# pointwise relative L2 error
Z_err = np.abs(Zp - Zt)                     # raw error
#Z_err_norm = Z_err / tau_l2_norm
#err_flat = Z_err[mask]              # flatten to disk
#L2_norm = np.sqrt(np.mean(err_flat**2))
#Z_err_norm = np.zeros_like(Z_err)
#Z_err_norm[mask] = Z_err[mask] / L2_norm

plt.figure(figsize=(6,5))
cf = plt.contourf(xx, yy, Z_err, levels=50, cmap='RdBu_r')
plt.colorbar(cf, label=r'$(|\tau_{PINN}-\tau|/\|\tau\|_{L^2}$')
plt.title('Normalized L²‐Error Field')
plt.axis('equal')
plt.tight_layout()
plt.show()